In [2]:
import sys

import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [3]:
df = hd.read_txn_data("777")
hd.inspect(df)

Time for read_txn_data                 : 45.83 seconds
(651,789, 21)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,is_urban,latest_balance,logins,postcode,region_name,tag_auto,yob
0,2012-02-01,777,400.00,<mdbremoved> - s/o,NaN,transfers,other_transfers,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688300,1.0,364.220001,0.0,wa1 4,north west,transfers,1969.0
1,2012-02-01,777,3.03,aviva pa - d/d,aviva,spend,finance,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688299,1.0,364.220001,0.0,wa1 4,north west,health insurance,1969.0


## Input data

Options and trade-offs:

- Full data-file

    - Either avoid user overlap
    
    - Or discard chunk endpoint users - this might be much simpler with minimal loss
    
- Use user-grouped files 0-9, but then can only use 10 cores


Decison:

- Our ec2 has 32 cores, would be silly to not use them. So use full file, and discard users on chunk ends (in analysis data, they will shop up as duplicated user rows, so simply 
